In [303]:
import json 
import requests 
import pickle

import pandas as pd
import numpy as np
import datetime, time

# import itertools
import warnings
warnings.filterwarnings('ignore')

In [304]:
# open data
health = pd.read_csv('D:/550Project/nyc_opendata/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

In [310]:
health.head().transpose()

,0,1,2,3,4
CAMIS,50115137,40711301,50131279,50074282,50119336
DBA,OMAR'S PLACE BK,CHICKEN FESTIVAL,NaN,BISMILAH RESTAURANT,COMMON GOOD HARLEM
BORO,Brooklyn,Queens,Manhattan,Brooklyn,Manhattan
BUILDING,1450,2919,355,2160,2801
STREET,MYRTLE AVENUE,DITMARS BOULEVARD,WEST 36 STREET,BATH AVENUE,FREDERICK DOUGLASS BOULEVARD
ZIPCODE,11237,11105,10018,11214,NaN
PHONE,3475623697,7187289696,3238280096,7184879592,9175290966
CUISINE DESCRIPTION,NaN,Chicken,NaN,Pakistani,Coffee/Tea
INSPECTION DATE,01/01/1900,02/07/2022,01/01/1900,03/25/2022,04/22/2022
ACTION,NaN,Violations were cited in the following area(s).,NaN,Violations were cited in the following area(s).,Violations were cited in the following area(s).


In [306]:
health.describe()

,CAMIS,ZIPCODE,SCORE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,Location Point
count,2.096280e+05,206689.000000,198713.000000,209330.000000,209330.000000,206154.000000,206154.000000,206154.000000,2.050610e+05,2.090670e+05,0.0
mean,4.710864e+07,10700.568555,21.742171,40.161474,-72.912074,252.103277,20.492734,29773.999137,2.548138e+06,2.435647e+09,NaN
std,4.203162e+06,593.975277,16.937126,4.771471,8.661916,129.393079,15.726227,31237.810890,1.338107e+06,1.330892e+09,NaN
min,3.007544e+07,7004.000000,0.000000,0.000000,-74.249101,101.000000,1.000000,100.000000,1.000000e+06,1.000000e+00,NaN
25%,4.156152e+07,10023.000000,11.000000,40.684034,-73.988814,106.000000,4.000000,7900.000000,1.049245e+06,1.010630e+09,NaN
50%,5.004852e+07,11004.000000,17.000000,40.730988,-73.956070,301.000000,20.000000,17500.000000,3.016149e+06,3.004430e+09,NaN
75%,5.009220e+07,11230.000000,29.000000,40.761787,-73.895841,401.000000,34.000000,42100.000000,4.002508e+06,4.001880e+09,NaN
max,5.013250e+07,14228.000000,157.000000,40.912822,0.000000,595.000000,51.000000,162100.000000,5.799501e+06,5.270001e+09,NaN


In [49]:
api_key = 'tfVORZDa17i92ZQO2apLx-O7nblcEy3gurlxq0vtcRTSHWEbeg1wvGJcJpicyRwA8FUdGJ3Kpxn_q7wdOpicbY7xa31ZpWqVhRfCiugBs1IlgtrfLU1axFgFuCbpY3Yx'

In [63]:
# URL to pull data from:
url = 'https://api.yelp.com/v3/businesses/search'

# Identify headers:
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [263]:
neighborhoods = sorted(['Midtown West', 'Greenwich Village', 'East Harlem', 'Upper East Side', 'Midtown East',
                 'Gramercy', 'Little Italy', 'Chinatown', 'SoHo', 'Harlem',
                 'Upper West Side', 'Tribeca', 'Garment District', 'Stuyvesant Town', 'Financial District',
                 'Chelsea', 'Morningside Heights', 'Times Square', 'Murray Hill', 'East Village',
                 'Lower East Side', 'Hell\s Kitchen', 'Central Park'])
neighborhoods = sorted(['Williamsburg','Park Slope','Brooklyn Heights','Bushwick','Dumbo','Greenpoint','Cobble Hill','Bayridge','Flatbush','Carroll Gardens'])
neighborhoods = sorted(['Astoria','Long Island City','Forest Hills','Sunnyside','Flushing','Ridgewood','Bayside','Jackson Heights','Jamaica','Rego Park','Elmhurst'])
neighborhoods = ['Battery Park City', 'Lower Manhattan','NoHo','Garment District','Meatpacking District', 'Lenox Hill','West Village','Flatiron District']

In [256]:
# Create temporary dataframe to hold data:
nyc = [[] for i in range(len(neighborhoods))] 

In [264]:
#Function to draw in data for each neighborhood:
for x in range(len(neighborhoods)):
    print('---------------------------------------------')
    print('Gathering Data for {}'.format(neighborhoods[x]))
    print('---------------------------------------------')


    for y in range(20):
        location = neighborhoods[x]+', Queens, NY'
        term = "Restaurant"
        search_limit = 50
        offset = 50 * y
        categories = "(restaurant, All)"
        sort_by = 'distance'
        radius = 10000

        url_params = {
                        'location': location.replace(' ', '+'),
                        'term' : term,
                        'limit': search_limit,
                        'offset': offset,
                        'categories': categories,
                        'sorty_by': sort_by,
                        'radius' : radius
                    }
        
        response = requests.get(url, headers=headers, params=url_params)
        print('***** {} Restaurants #{} - #{} ....{}'.format(neighborhoods[x], 
                                                             offset+1, offset+search_limit,
                                                             response))
        nyc[x].append(response)

print(response)
print(type(response.text))
print(response.json().keys())
#print(response.text[:1000])

---------------------------------------------
Gathering Data for Battery Park City
---------------------------------------------
***** Battery Park City Restaurants #1 - #50 ....<Response [200]>
***** Battery Park City Restaurants #51 - #100 ....<Response [200]>
***** Battery Park City Restaurants #101 - #150 ....<Response [200]>
***** Battery Park City Restaurants #151 - #200 ....<Response [200]>
***** Battery Park City Restaurants #201 - #250 ....<Response [200]>
***** Battery Park City Restaurants #251 - #300 ....<Response [200]>
***** Battery Park City Restaurants #301 - #350 ....<Response [200]>
***** Battery Park City Restaurants #351 - #400 ....<Response [200]>
***** Battery Park City Restaurants #401 - #450 ....<Response [200]>
***** Battery Park City Restaurants #451 - #500 ....<Response [200]>
***** Battery Park City Restaurants #501 - #550 ....<Response [200]>
***** Battery Park City Restaurants #551 - #600 ....<Response [200]>
***** Battery Park City Restaurants #601 - #650

***** Lenox Hill Restaurants #701 - #750 ....<Response [200]>
***** Lenox Hill Restaurants #751 - #800 ....<Response [200]>
***** Lenox Hill Restaurants #801 - #850 ....<Response [200]>
***** Lenox Hill Restaurants #851 - #900 ....<Response [200]>
***** Lenox Hill Restaurants #901 - #950 ....<Response [200]>
***** Lenox Hill Restaurants #951 - #1000 ....<Response [200]>
---------------------------------------------
Gathering Data for West Village
---------------------------------------------
***** West Village Restaurants #1 - #50 ....<Response [200]>
***** West Village Restaurants #51 - #100 ....<Response [200]>
***** West Village Restaurants #101 - #150 ....<Response [200]>
***** West Village Restaurants #151 - #200 ....<Response [200]>
***** West Village Restaurants #201 - #250 ....<Response [200]>
***** West Village Restaurants #251 - #300 ....<Response [200]>
***** West Village Restaurants #301 - #350 ....<Response [200]>
***** West Village Restaurants #351 - #400 ....<Response [2

In [265]:
df = pd.DataFrame()
for x in range(len(neighborhoods)):
    for y in range(20):
        if (str(nyc[x][y]) == '<Response [200]>') and (nyc[x][y].json()['businesses'] != []):
            df_temp = pd.DataFrame.from_dict(nyc[x][y].json()['businesses'])
            df_temp.loc[:,'neighborhood'] = neighborhoods[x]
            df = df.append(df_temp)
df.head()

In [267]:
# Save Dataset: (data pulled 8/25/20)
# with open ('data_restaurants.pickle','wb')as f:
#     pickle.dump(df, f)
# with open ('data_bars.pickle','wb')as f:
#      pickle.dump(df, f)
# with open ('data_fastfood.pickle','wb')as f:
#      pickle.dump(df, f)
# with open ('data_brooklyn.pickle','wb')as f:
#      pickle.dump(df, f)
# with open ('data_queens.pickle','wb')as f:
#      pickle.dump(df, f)
# with open ('data_manhattan_p2.pickle','wb')as f:
#      pickle.dump(df, f)

In [268]:
with open ('data_restaurants.pickle','rb') as f:
    df1 = pickle.load(f)
with open ('data_bars.pickle','rb') as f:
    df2 = pickle.load(f)
with open ('data_fastfood.pickle','rb') as f:
    df3 = pickle.load(f)
with open ('data_brooklyn.pickle','rb') as f:
    df4 = pickle.load(f)
with open ('data_queens.pickle','rb') as f:
    df5 = pickle.load(f)
with open ('data_manhattan_p2.pickle','rb') as f:
    df6 = pickle.load(f)

In [300]:
df = df1.append(df2).append(df3).append(df4).append(df5).append(df6)
len(df.id.unique())

12632

In [276]:
# save final
# df_finals = df.sort_values(['id','distance']).groupby('id').head(1)
# with open ('yelp_nyc.pickle','wb')as f:
#      pickle.dump(df_finals, f)
# df_finals.to_csv('yelp_nyc.csv')

In [307]:
df_finals.describe()

,review_count,rating,distance
count,12632.000000,12632.000000,12632.000000
mean,238.495329,3.863205,1413.992681
std,471.422530,0.851894,1626.063806
min,0.000000,0.000000,10.312452
25%,20.000000,3.500000,405.521847
50%,87.000000,4.000000,743.716938
75%,270.000000,4.500000,1892.220854
max,14666.000000,5.000000,11971.819226


In [309]:
df_finals.head().transpose()

,11,28,14,30,35
id,--FoSeUBHQb_p6iTntY4KA,--SoNTeCXFUx56no0OgibQ,--YBn9-xi4YocUK6VvCKqQ,--ZEJucmNxRxdZ0T-YkR3w,-0bdnX762vdTb9lI00etvA
alias,stafili-wine-cafe-new-york,bites-of-xian-new-york,the-local-astoria-2,lechonera-pollo-sabroso-new-york,brooklyn-burgers-and-beer-brooklyn
name,Stafili Wine Cafe,Bites of Xi'an,The Local,Lechonera Pollo Sabroso,Brooklyn Burgers & Beer
image_url,https://s3-media4.fl.yelpcdn.com/bphoto/aBpSo_...,https://s3-media3.fl.yelpcdn.com/bphoto/TIqRwE...,https://s3-media2.fl.yelpcdn.com/bphoto/u34kfw...,https://s3-media4.fl.yelpcdn.com/bphoto/XmdFkS...,https://s3-media3.fl.yelpcdn.com/bphoto/Xt6uxV...
is_closed,False,False,False,False,False
url,https://www.yelp.com/biz/stafili-wine-cafe-new...,https://www.yelp.com/biz/bites-of-xian-new-yor...,https://www.yelp.com/biz/the-local-astoria-2?a...,https://www.yelp.com/biz/lechonera-pollo-sabro...,https://www.yelp.com/biz/brooklyn-burgers-and-...
review_count,45,78,58,8,432
categories,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","[{'alias': 'divebars', 'title': 'Dive Bars'}, ...","[{'alias': 'latin', 'title': 'Latin American'}...","[{'alias': 'burgers', 'title': 'Burgers'}, {'a..."
rating,5,4.5,4.5,3,4
coordinates,"{'latitude': 40.73740497548472, 'longitude': -...","{'latitude': 40.730861, 'longitude': -73.989941}","{'latitude': 40.7607294903307, 'longitude': -7...","{'latitude': 40.82694, 'longitude': -73.92513}","{'latitude': 40.6743507, 'longitude': -73.9815..."


In [290]:
# to fetch reviews, api only allows 3 per business now

url = 'https://api.yelp.com/v3/businesses/zRXMvxUX_rOliKZPpkWi_g/reviews'

url_params = {
                'id':'zRXMvxUX_rOliKZPpkWi_g',
                'sort_by':'yelp_sort',
                'limit':3
            }

response = requests.get(url, headers=headers, params=url_params)
response

<Response [200]>

In [299]:
pd.DataFrame.from_dict(response.json()['reviews'])

,id,url,text,rating,time_created,user
0,r8lq046zt06FvD2acwCrVg,https://www.yelp.com/biz/valerie-new-york?adju...,Amazing vibe. Staff is SO kind and helpful. Go...,5,2022-12-27 11:19:41,"{'id': 'x6AyjMi32JiOxHPuT_a2Tg', 'profile_url'..."
1,29nyTgMOL6Anf4055rj-Dw,https://www.yelp.com/biz/valerie-new-york?adju...,Love this place for happy hour!! This place ge...,5,2023-01-27 20:33:03,"{'id': '7i6kDFUbMVXQwNz9WcEV7Q', 'profile_url'..."
2,0uVJbvTPQ9Q5ZA5qGyXLGA,https://www.yelp.com/biz/valerie-new-york?adju...,A romantic spot for a first date. Nice ambianc...,4,2023-02-04 14:22:45,"{'id': '_Yq7t2c-4miIFGmaUssMwg', 'profile_url'..."
